In [2]:
print("ok")

ok


In [6]:
%pwd

'/Users/anishneunaha/Desktop/GGU/Assignments/Group Project/VitalMediSync/VitalMediSync'

In [5]:
import os
os.chdir("../")

In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# Extract the data from the PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents


In [9]:
extracted_data = load_pdf_file(data='data/')

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print(f"Length of Text Chunks: {len(text_chunks)}")

Length of Text Chunks: 39994


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

In [18]:
query_result = embeddings.embed_query("Hello World")
print(f"Length : {len(query_result)}")

Length : 384


In [33]:
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "vitalmedisync"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [24]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [27]:
# Embedd each chunk and insert the chunks into the Pinecone Index
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
)

In [28]:
#Load exisiting index

docsearch = Pinecone.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [31]:
retrieved_docs = retriever.invoke("What is Diabetes?")

In [32]:
retrieved_docs

[Document(metadata={'page': 1184.0, 'source': 'data/medical_book.pdf'}, page_content='Description\nDiabetes mellitus is a chronic disease that causes\nserious health complications including renal (kidney)\nfailure, heart disease,stroke, and blindness.\nApproximately 17 million Americans have diabetes.\nUnfortunately, as many as one-half are unaware they\nhave it.\nBackground\nEvery cell in the human body needs energy in\norder to function. The body’s primary energy source\nis glucose, a simple sugar resulting from the digestion\nof foods containing carbohydrates (sugars and'),
 Document(metadata={'page': 1184.0, 'source': 'data/medical_book.pdf'}, page_content='Diabetes mellitus\nDefinition\nDiabetes mellitus is a condition in which the pan-\ncreas no longer produces enough insulin or cells stop\nresponding to the insulin that is produced, so that\nglucose in the blood cannot be absorbed into the\ncells of the body. Symptoms include frequent urina-\ntion, lethargy, excessive thirst, an

In [35]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=1000)

In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [38]:
system_prompt =(
    "You are a conversational Assistant whose responsibility is to act like a companion and a medical professional to provide answers to your patients."
    "Ensure that you only provide responses based on the retrieved context to answer the questions. If you do not know any specific answer then ensure that you inform the patient to consult his/her medical practioner for better safety."
    "Use upto 5 sentences to answer the query, make sure you to sympathetic, supportive and kind while answering"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}")
    ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [40]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])



System: Acne is a common skin disease that is characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is caused by a variety of factors, including genetics, hormones, and certain medications. Treatment for acne may involve the use of topical or oral medications, depending on the severity of the condition. It is important to consult with a medical professional for proper diagnosis and treatment.


In [41]:
response = rag_chain.invoke({"input": "What is Diabetes?"})
print(response["answer"])



System: Diabetes is a chronic disease that affects millions of Americans. It is caused by the body's inability to produce enough insulin or respond to the insulin that is produced, resulting in high levels of glucose in the blood. This can lead to serious health complications, such as kidney failure, heart disease, stroke, and blindness. It is important to be aware of the symptoms, including frequent urination, lethargy, excessive thirst, and hunger, and to seek treatment, which may include changes in diet, oral medications, and insulin injections. If you have any concerns or experience these symptoms, it is best to consult your medical practitioner for proper diagnosis and treatment. Remember, you are not alone and there are ways to manage and live with diabetes.


In [42]:
response = rag_chain.invoke({"input": "What are they symptoms of fever?"})
print(response["answer"])



System: The symptoms of fever can vary depending on the underlying cause. However, common symptoms include a high body temperature, chills, sweating, headache, muscle and joint pain, weakness, nausea, vomiting, and sensitivity to light. In severe cases, the patient may also experience confusion, dizziness, and swollen organs. If you are experiencing these symptoms, it is important to consult your medical practitioner for proper diagnosis and treatment. Take care and stay safe.
